In [17]:
## Run when initialise the code


import pandas as pd
from pandas import DataFrame
import geopandas as gpd

from geopandas import GeoDataFrame
from shapely.geometry import Point, MultiPolygon,LineString, MultiLineString
from shapely.ops import linemerge, unary_union,split
from shapely.strtree import STRtree
from shapely.geometry.base import BaseGeometry
from shapely.prepared import prep

import os
import requests
from tqdm import tqdm
from sklearn.cluster import DBSCAN

import numpy as np
import math
from math import log2

from itertools import groupby,combinations
from collections import defaultdict

import warnings
warnings.filterwarnings(action='ignore')

pjr_loc = os.path.dirname(os.getcwd())
project_crs = 'epsg:3857'
place = 'Turin,Italy'
place_folder = f'{pjr_loc}/places'
os.makedirs(place_folder , exist_ok=True)
data_folder = f'{place_folder}/{place.replace(",","_")}'
os.makedirs(data_folder, exist_ok=True)
test_folder = f'{data_folder}/test'
os.makedirs(test_folder, exist_ok=True)

In [20]:


def get_bbox_from_nominatim(place_name):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": place_name, "format": "json", "limit": 1}
    headers = {
        "User-Agent": "GeoResearcher/1.0 (achituv@ariel.ac.il)"  # Replace with your email
    }
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()
    results = r.json()
    if not results:
        raise ValueError(f"No results found for place: {place_name}")
    bbox = results[0]["boundingbox"]
    return float(bbox[0]), float(bbox[1]), float(bbox[2]), float(bbox[3])

def download_osm_roads_bbox(place_name, highway_tags=None):
    if highway_tags is None:
        highway_tags = [ 'primary','footway','pedestrian','cycleway', 'path','secondary', 'tertiary',
                        'unclassified', 'residential', 'service', 'living_street','steps']
    south, north, west, east = get_bbox_from_nominatim(place_name)
    highway_filter = '|'.join(highway_tags)

    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json][timeout:180];
    (
      way["highway"~"{highway_filter}"]({south},{west},{north},{east});
    );
    out geom;
    """

    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    roads = []
    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(p['lon'], p['lat']) for p in element['geometry']]
            if len(coords) >= 2:
                geom = LineString(coords)
                tags = element.get('tags', {})
                tags['geometry'] = geom
                tags['osmid'] = element['id']
                roads.append(tags)

    gdf = gpd.GeoDataFrame(roads, geometry='geometry', crs='EPSG:4326')
    return gdf

def remove_unconnected_streets(df_connected, str_name='name_left', con_str_name='name_right'):
    """
    Remove streets from a GeoDataFrame that are not connected to any other streets.

    Parameters:
    df_connected (GeoDataFrame): The input GeoDataFrame containing street data.
    str_name (str): The column name for the street name in the first dataset.
    con_str_name (str): The column name for the connected street name in the second dataset.

    Returns:
    GeoDataFrame: A new GeoDataFrame with unconnected streets removed.
    """
    df_analysis = df_connected.copy()
    s_join_analysis = gpd.sjoin(df_analysis, df_connected)
    s_join_analysis2 = s_join_analysis[s_join_analysis[str_name] != s_join_analysis[con_str_name]]
    not_connected = set(df_connected['name']) - set(s_join_analysis2.reset_index()[str_name])
    df_pro_filtered = df_connected[~df_connected['name'].isin(not_connected)]
    return df_pro_filtered
def calculate_bearing(line):
    if line is None or line.is_empty or len(line.coords) < 2:
        return np.nan

    start, end = line.coords[0], line.coords[-1]
    dx = end[0] - start[0]
    dy = end[1] - start[1]

    angle = np.degrees(np.arctan2(dx, dy))
    bearing = (angle + 360) % 360
    return bearing


In [21]:
gdf_roads = download_osm_roads_bbox(place)[['highway','name','osmid','junction','geometry']].to_crs(project_crs )
gdf_roads.to_file(f"{data_folder}/osm_data.shp")
# Create preprocessing object and folder
gdf_roads = remove_unconnected_streets(gdf_roads)
roundabout = gdf_roads[gdf_roads['junction'].isin(['roundabout', 'circular','mini_roundabout'])]
gdf_edges = gdf_roads[~gdf_roads['junction'].isin(['roundabout', 'circular','mini_roundabout'])]
# Load and filter the edges from the saved GeoPackage

gdf_edges = gdf_roads[~gdf_roads['highway'].str.contains('link', case=False, na=False)]
gdf_edges = gdf_edges.dropna(subset=['name'])
gdf_edges = gdf_edges[gdf_edges['name'] != '']

gdf_edges['bearing'] = gdf_edges['geometry'].apply(calculate_bearing)
gdf_edges['angle'] = gdf_edges['bearing'].apply(lambda x: x if x < 180 else x - 180)
gdf_edges['length'] = gdf_edges.length
gdf_edges.to_file(f'{data_folder}/edges.shp')

In [22]:
def extract_deadend_points(gdf_edges):
    """
    Identifies all dead-end points (start or end points that appear only once in the entire edge dataset).

    Parameters:
        gdf_edges (GeoDataFrame): Edge lines with LineString geometries.

    Returns:
        Set of Points: All dead-end points.
    """
    endpoint_counter = defaultdict(int)

    for geom in gdf_edges.geometry:
        coords = list(geom.coords)
        endpoint_counter[coords[0]] += 1
        endpoint_counter[coords[-1]] += 1
    return endpoint_counter


In [23]:
def build_street_connection_dict(gdf_streets):
    """
    Build a dictionary of street-to-street connections via unique intersection points.

    Parameters:
        gdf_streets (GeoDataFrame): A GeoDataFrame containing street segments, with a 'name' column.

    Returns:
        connection_dict (dict): A dictionary where each key is a tuple of two street names (str, str),
                                and each value is a list of unique shapely Points where the streets intersect.
    """
    # Ensure clean index
    gdf = gdf_streets.reset_index(drop=True)

    # Step 1: Perform spatial join to find intersecting features
    joined = gpd.sjoin(gdf, gdf, how='inner', predicate='intersects')
    joined = joined[joined.index != joined['index_right']]  # Remove self-intersections

    # Step 2: Initialize output containers
    connection_dict = defaultdict(list)
    seen_edges = set()

    # Step 3: Process each intersecting pair
    for idx, row in tqdm(joined.iterrows(), total=len(joined)):
        idx2 = row['index_right']
        name1 = row['name_left']
        name2 = row['name_right']

        # Skip connections within the same street
        if name1 == name2:
            continue

        pair_key = tuple(sorted((name1, name2)))
        geom1 = row.geometry
        geom2 = gdf.loc[idx2].geometry
        intersection = geom1.intersection(geom2)

        # Skip if no actual intersection
        if intersection.is_empty:
            continue

        # Normalize intersection geometry into list of points
        if intersection.geom_type == 'Point':
            inter_points = [intersection]
        elif intersection.geom_type == 'MultiPoint':
            inter_points = list(intersection.geoms)
        else:
            continue  # Ignore non-point intersections

        # Step 4: Store only new intersection points
        existing_coords = {tuple(pt.coords)[0] for pt in connection_dict[pair_key]}
        new_points = [pt for pt in inter_points if tuple(pt.coords)[0] not in existing_coords]

        if new_points:
            connection_dict[pair_key].extend(new_points)
            seen_edges.add(pair_key)

    return connection_dict
conn_dict = build_street_connection_dict(gdf_edges)

100%|██████████| 76592/76592 [00:06<00:00, 11854.59it/s]


In [24]:
# Merge roundabout geometries, identify which streets intersect them, and extract roundabout center points for further spatial analysis.


# ------------------------------------------------------------
# STEP 1: Merge roundabout edges into clean geometries
# - Union geometries to dissolve boundaries
# - Linemerge to connect continuous segments
# - Export merged geometries as shapefile
# ------------------------------------------------------------
roundabout_edges = roundabout  # GeoDataFrame with roundabout segments

# Merge all geometries into a MultiLineString
merged_geom = unary_union(roundabout_edges.geometry)

# Connect continuous line segments
connected_lines = linemerge(merged_geom)

# Normalize to list of LineStrings
if isinstance(connected_lines, LineString):
    lines_list = [connected_lines]
elif isinstance(connected_lines, MultiLineString):
    lines_list = list(connected_lines.geoms)
else:
    lines_list = []

# Save merged roundabout geometries for inspection
gdf_roundabout = gpd.GeoDataFrame(geometry=lines_list, crs=roundabout_edges.crs)
gdf_roundabout.to_file(f'{data_folder}/roundabout.shp')

# ------------------------------------------------------------
# STEP 2: Create a buffer around merged roundabouts and find intersecting street edges
# ------------------------------------------------------------
# Buffer each merged roundabout line (e.g., 5 meters)
roundabout_buffer = gdf_roundabout.copy()
roundabout_buffer['geometry'] = roundabout_buffer.geometry.buffer(5)

# Perform spatial join: find street edges that intersect the roundabout buffer
intersections = gpd.sjoin(gdf_edges, roundabout_buffer, how='inner', predicate='intersects')

# ------------------------------------------------------------
# STEP 3: Build dictionary mapping roundabout index to connected street names
# ------------------------------------------------------------
roundabout_to_streets = defaultdict(set)

for _, row in intersections.iterrows():
    roundabout_idx = row['index_right']  # index of the matched roundabout in gdf_merged
    street_name = row['name']
    roundabout_to_streets[roundabout_idx].add(street_name)

# Save the original roundabout edge segments for reference
roundabout_edges.to_file(f'{data_folder}/roundabout_edges.shp')

# ------------------------------------------------------------
# STEP 4: Calculate center point for each roundabout polygon buffer
# - Buffer again with clean style, then extract centroid
# ------------------------------------------------------------
# Buffer with styling for rounded shapes, then get centroids
roundabout_centers = roundabout_buffer['geometry'].buffer(distance=1, cap_style=1, join_style=1).centroid

# Compute distance from centroid to the boundary of the buffer
distances = [
    centroid.distance(buffer.boundary)
    for centroid, buffer in zip(roundabout_centers, roundabout_buffer['geometry'])
]

# Save as GeoDataFrame
gdf_roundabouts = gpd.GeoDataFrame(geometry=roundabout_centers, crs=roundabout_edges.crs)
# Compute distance from centroid to buffer boundary
gdf_roundabouts['dist'] = distances
gdf_roundabouts.reset_index(drop=True).to_file(f'{data_folder}/center_roundabout.shp')



In [25]:

# This code performs the main part of the simplification
# --- Helper Functions ---

def check_parallelism(to_translate: GeoDataFrame) -> bool:
    """
    Checks whether a group of line segments contain any parallel segments
    by offsetting each line and checking buffer intersections.
    """
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'left'))

    def is_parallel(my_s_join: GeoDataFrame, the_buffer, geo_field: str):
        my_s_join['geometry'] = my_s_join[geo_field]
        sjoin = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs=project_crs), how='inner')
        sjoin = sjoin[sjoin.index != sjoin['index_right']]
        for _, row in sjoin.iterrows():
            overlay = gpd.overlay(
                GeoDataFrame([row], crs=project_crs),
                GeoDataFrame(geometry=[the_buffer[row['index_right']]], crs=project_crs),
                how='intersection')
            if (overlay.length / row.geometry.length).iloc[0] * 100 > 10:
                return True
        return False

    return is_parallel(to_translate, my_buffer, 'geometry_right') or is_parallel(to_translate, my_buffer, 'geometry_left')

def circular_distance(angle1, angle2):
    """Compute minimum circular angle difference between two angles."""
    diff = np.abs(angle1 - angle2) % 180
    return np.minimum(diff, 180 - diff)

def add_more_pnts_to_new_lines(pnt_f, pnt_l, line_pnts, lngth_chck, test_poly):
    """
    Recursively add points along a line if they're more than 10m from existing road segments.
    """
    dist = pnt_f.distance(pnt_l)
    x0, y0 = pnt_f.x, pnt_f.y
    bearing = math.atan2(pnt_l.x - x0, pnt_l.y - y0)
    if bearing < 0:
        bearing += 2 * math.pi
    loops = int(dist / lngth_chck)

    for step in range(1, loops):
        x_new = x0 + lngth_chck * step * math.sin(bearing)
        y_new = y0 + lngth_chck * step * math.cos(bearing)
        new_point = Point(x_new, y_new)
        nearest = GeoDataFrame(geometry=[new_point], crs=project_crs).sjoin_nearest(test_poly, distance_col='dis').iloc[0]
        if nearest['dis'] > 10:
            line = data.loc[nearest['index_right']]['geometry']
            projected = line.interpolate(line.project(new_point))
            if projected.distance(pnt_f) < 10:
                continue
            line_pnts.append(projected)
            return add_more_pnts_to_new_lines(projected, pnt_l, line_pnts, lngth_chck, test_poly)
    return line_pnts

def create_center_line(one_poly):
    """
    Construct a center line through a polygon by identifying its farthest endpoints
    and interpolating new points as needed based on angular continuity.
    """
    lines = data.sjoin(GeoDataFrame(geometry=[one_poly], crs=project_crs)).drop(columns='index_right')
    endpoints = []
    lines['geometry'].apply(lambda line: endpoints.extend([Point(line.coords[0]), Point(line.coords[-1])]))
    combos = list(combinations(endpoints, 2))

    df = DataFrame({
        'point_1': [a for a, _ in combos],
        'point_2': [b for _, b in combos],
    })
    df['dist'] = df.apply(lambda x: x['point_1'].distance(x['point_2']), axis=1)
    dx = df['point_2'].apply(lambda p: p.x) - df['point_1'].apply(lambda p: p.x)
    dy = df['point_2'].apply(lambda p: p.y) - df['point_1'].apply(lambda p: p.y)
    df['angle'] = np.degrees(np.arctan2(dy, dx)) % 180
    avg_angle = lines['angle'].mean()
    df['ratio'] = df['dist'] / df['dist'].max() + 0.5 * np.abs(df['angle'] - avg_angle) / np.abs(df['angle'] - avg_angle).max()

    pnt_f, pnt_l = df.sort_values(by='ratio', ascending=False).iloc[0][['point_1', 'point_2']]
    angle_range = lines['angle'].max() - lines['angle'].min()

    if angle_range < 1:
        new_line_pts = [pnt_f]
    else:
        step = 8.5 if angle_range > 100 else 75 - log2(angle_range) * 10
        new_line_pts = add_more_pnts_to_new_lines(pnt_f, pnt_l, [pnt_f], step, lines)
    new_line_pts.append(pnt_l)
    return new_line_pts

def update_df_with_center_line(new_line, is_simplified=0, group_name=-1):
    """Append a new line feature to the output dictionary."""
    dic_final['name'].append(name)
    dic_final['geometry'].append(new_line)
    dic_final['highway'].append(data.iloc[0]['highway'])
    dic_final['bearing'].append(data['angle'].mean())
    dic_final['group'].append(group_name)
    dic_final['is_simplified'].append(is_simplified)

# --- Main logic ---
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': [], 'group': [], 'is_simplified': []}
df_pro = gdf_edges
grouped = df_pro.groupby('name')

with tqdm(total=len(grouped)) as pbar:
    for name, group_df in grouped:
        pbar.update(1)
        group_df = group_df.dropna(subset=['angle'])
        if len(group_df) < 2:
            data = group_df
            _ = group_df['geometry'].apply(lambda geom: update_df_with_center_line(geom))
            continue

        angles = group_df['angle'].to_numpy()
        dists = np.array([[circular_distance(a1, a2) for a2 in angles] for a1 in angles])
        dbscan = DBSCAN(eps=10, min_samples=2, metric='precomputed')
        group_df['group'] = dbscan.fit_predict(dists)

        if (group_df['group'] == -1).all():
            data = group_df
            _ = group_df['geometry'].apply(lambda geom: update_df_with_center_line(geom))
            continue

        for group_id, sub_group in group_df.groupby('group'):
            data = sub_group
            if group_id == -1:
                _ = data['geometry'].apply(lambda geom: update_df_with_center_line(geom))
                continue
            if check_parallelism(data.copy()):
                min_polylines = len(data) / 15
                condition = (data['highway'].isin(['service', 'unclassified'])) & (
                    data.groupby('highway')['highway'].transform('count') <= min_polylines)
                data = data[~condition]

                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                unified = buffers.unary_union

                if isinstance(unified, MultiPolygon):
                    for poly in unified.geoms:
                        center_pts = create_center_line(poly)
                        update_df_with_center_line(LineString(center_pts), 1, group_id)
                else:
                    center_pts = create_center_line(unified)
                    update_df_with_center_line(LineString(center_pts), 1, group_id)
            else:
                _ = data['geometry'].apply(lambda geom: update_df_with_center_line(geom))

# Finalize and export
print(f'number_of_parallel: {sum(dic_final["is_simplified"])}')
print('create new files')
new_network = GeoDataFrame(dic_final, crs=project_crs)
new_network['length'] = new_network.length
new_network.to_file(f'{data_folder}/simp.shp')


100%|██████████| 3193/3193 [02:33<00:00, 20.78it/s] 


number_of_parallel: 1306
create new files


In [26]:
# The code filters out short, weakly connected street segments that don't intersect other lines at their endpoints and cleans up the geometry for a simplified street network.

# Copy the simplified network
gdf = new_network.copy()

# ----------------------------------------------------------------
# Step 1: Precompute useful attributes
# ----------------------------------------------------------------

# Count how many times each street name appears
gdf['name_count'] = gdf['name'].map(gdf['name'].value_counts())

# Count how many times each endpoint appears (used for degree check)
endpoint_counts = defaultdict(int)
for _, row in gdf.iterrows():
    coords = list(row.geometry.coords)
    endpoint_counts[coords[0]] += 1
    endpoint_counts[coords[-1]] += 1

# ----------------------------------------------------------------
# Step 2: Keep only lines from streets that are connected to roundabouts
# -----------------------------
# -----------------------------------
connected_streets = {name for street_set in roundabout_to_streets.values() for name in street_set}
gdf_connected_0 = gdf[gdf['name'].isin(connected_streets)].copy()
gdf_connected = gdf_connected_0[gdf_connected_0.length>30]
lines_to_remove  = set(gdf_connected_0[gdf_connected_0.geometry.length < 30].index)
# ----------------------------------------------------------------
# Step 3: Identify candidate short terminal lines
# ----------------------------------------------------------------
candidates = gdf_connected[
    (gdf_connected['length'] < 100) &
    (gdf_connected['name_count'] > 1) &
    (gdf_connected.geometry.apply(lambda geom: endpoint_counts[geom.coords[0]] == 1 and
                                                 endpoint_counts[geom.coords[-1]] == 1))
].copy()

# ----------------------------------------------------------------
# Step 4: Check intersections of candidates with other edges
# ----------------------------------------------------------------
intersections = gpd.sjoin(candidates, gdf, how='inner', predicate='intersects')
intersections = intersections[intersections.index != intersections['index_right']]  # exclude self-intersections

# ----------------------------------------------------------------
# Step 5: Determine which candidates should be removed
# ----------------------------------------------------------------

for idx, row in candidates.iterrows():
    temp_line = row.geometry
    start = Point(temp_line.coords[0])
    end = Point(temp_line.coords[-1])

    # All intersections involving this line
    matching = intersections.loc[intersections.index == idx]

    # If there are no intersections at all, remove the line
    if matching.empty:
        lines_to_remove.add(idx)
        continue

    # Check whether any intersection point lies exactly at start or end
    valid_intersection = False
    for other_idx in matching['index_right']:
        other_line = gdf.loc[other_idx].geometry
        inter = temp_line.intersection(other_line)

        if inter.is_empty:
            continue

        # Ensure we're working with a list of Points
        inter_points = [inter] if isinstance(inter, Point) else list(inter.geoms)

        # Check if any intersecting point matches start or end
        if any(pt.equals(start) or pt.equals(end) for pt in inter_points):
            valid_intersection = True
            break

    if not valid_intersection:
        lines_to_remove.add(idx)

# ----------------------------------------------------------------
# Step 6: Remove flagged lines and clean up geometries
# ----------------------------------------------------------------
gdf_simplified = gdf.drop(index=lines_to_remove).copy()



# Remove consecutive duplicate coordinates
gdf_simplified['geometry'] = gdf_simplified['geometry'].apply(
    lambda geom: LineString([pt for pt, _ in groupby(geom.coords)]) if len(set(geom.coords)) > 1 else None
)


# Drop invalid geometries (empty or single-point lines)
gdf_simplified = gdf_simplified[gdf_simplified['geometry'].notnull()].copy()
gdf_simplified.to_file(f'{data_folder}/test/gdf_simplified_0.shp')




In [27]:


# --- Parameters ---
TOLERANCE = 20
TOLERANCE_NEAR = 50
updated_geoms = {}
gdf_simplified_fix1 = gdf_simplified.copy()

# --- Precompute line coordinates and cache results ---
line_geoms = gdf_simplified_fix1.geometry.values
line_coords = [list(geom.coords) for geom in line_geoms]
gdf_simplified_fix1['coords'] = line_coords

# --- Step 1: Iterate roundabouts and connected streets ---
for ridx, streets in roundabout_to_streets.items():
    roundabout_point = gdf_roundabouts.geometry.iloc[ridx]

    for street in streets:
        street_edges = gdf_simplified_fix1[gdf_simplified_fix1['name'] == street]

        if street_edges.empty:
            continue

        # Vectorized distance calc for all candidate edges
        dists = street_edges.geometry.apply(lambda g: g.distance(roundabout_point))
        min_dist = dists.min()
        closest_edges = street_edges[dists == min_dist]

        for edge_idx, edge_row in closest_edges.iterrows():
            line = updated_geoms.pop(edge_idx, edge_row.geometry)
            coords = list(line.coords)

            nearest_pt = line.interpolate(line.project(roundabout_point))

            dist_to_start = nearest_pt.distance(Point(coords[0]))
            dist_to_end = nearest_pt.distance(Point(coords[-1]))

            # --- Insertion Logic ---
            if dist_to_start > TOLERANCE and dist_to_end > TOLERANCE:
                insert_idx = np.argmin([Point(c).distance(nearest_pt) for c in coords])

                round_coord = roundabout_point.coords[0]
                if insert_idx == 0:
                    new_coords = [round_coord] + coords[1:] if roundabout_point.distance(Point(coords[0])) < TOLERANCE_NEAR else [round_coord] + coords
                elif insert_idx == len(coords) - 1:
                    new_coords = coords[:-1] + [round_coord] if roundabout_point.distance(Point(coords[-1])) < TOLERANCE_NEAR else coords[:-1] + [round_coord] + [coords[-1]]
                else:
                    if roundabout_point.distance(Point(coords[insert_idx])) < TOLERANCE_NEAR:
                        new_coords = coords[:insert_idx] + [round_coord] + coords[insert_idx + 1:]
                    else:
                        def bearing(p1, p2):
                            dx, dy = p2[0] - p1[0], p2[1] - p1[1]
                            return np.degrees(np.arctan2(dy, dx)) % 360
                        az1 = bearing(coords[insert_idx - 1], coords[insert_idx])
                        az2 = bearing(coords[insert_idx], round_coord)
                        diff = abs((az1 - az2 + 180) % 360 - 180)
                        if 150 <= diff <= 210:
                            new_coords = coords[:insert_idx] + [round_coord] + coords[insert_idx:]
                        else:
                            new_coords = coords[:insert_idx + 1] + [round_coord] + coords[insert_idx + 1:]
            else:
                new_coords = [roundabout_point.coords[0]] + coords[1:] if dist_to_start < dist_to_end else coords[:-1] + [roundabout_point.coords[0]]

            updated_geoms[edge_idx] = LineString(new_coords)

# --- Step 3: Batch apply updates ---
gdf_simplified_fix1.loc[updated_geoms.keys(), 'geometry'] = list(updated_geoms.values())
gdf_simplified_fix1.drop(columns='coords', inplace=True, errors='ignore')


In [28]:
# This code snaps dead-end street endpoints to nearby roundabouts if they're within a specified distance buffer, improving network connectivity.

# Make a working copy
gdf_simplified_fix2 = gdf_simplified_fix1.copy()

# --- Parameters ---

updated_geoms_buffer = {}

# --- Step 1: Build node appearance count, including roundabout centers ---
node_count = defaultdict(int)

# Count start and end points of all line segments
for _, row in gdf_simplified_fix2.iterrows():
    coords = list(row.geometry.coords)
    node_count[coords[0]] += 1
    node_count[coords[-1]] += 1

# Also count roundabout center points
for pt in gdf_roundabouts.geometry:
    node_count[pt.coords[0]] += 1

# --- Helper: Check if roundabout already in line ---
def is_roundabout_in_line(roundabout_point, line_coords):
    return roundabout_point.coords[0] in line_coords

# --- Step 2: Snap dead-end line endpoints to roundabout if within buffer ---
for ridx, streets in roundabout_to_streets.items():
    roundabout_row = gdf_roundabouts.loc[ridx]
    roundabout_point = roundabout_row.geometry
    roundabout_buffer = roundabout_point.buffer(roundabout_row['dist']*2)

    for street in streets:
        street_edges = gdf_simplified_fix2[gdf_simplified_fix2['name'] == street]

        for idx, row in street_edges.iterrows():
            # Use previously updated geometry if exists
            coords = list(updated_geoms_buffer.pop(idx, row.geometry).coords)

            # Only adjust if roundabout not already in the line
            if not is_roundabout_in_line(roundabout_point, coords):
                start, end = Point(coords[0]), Point(coords[-1])

                # Snap start if it's a dead-end and inside buffer
                if node_count[coords[0]] == 1 and roundabout_buffer.contains(start):
                    coords[0] = roundabout_point.coords[0]

                # Snap end if it's a dead-end and inside buffer
                elif node_count[coords[-1]] == 1 and roundabout_buffer.contains(end):
                    coords[-1] = roundabout_point.coords[0]

            # Save updated geometry
            updated_geoms_buffer[idx] = LineString(coords)

# --- Step 3: Apply changes to GeoDataFrame ---
for idx, geom in updated_geoms_buffer.items():
    gdf_simplified_fix2.at[idx, 'geometry'] = geom



In [29]:
# This code removes duplicate street segments (with same endpoints but different geometries) around roundabouts, keeping only the longest one per pair.


# Make a working copy of the dataset
gdf_simplified_fix3 = gdf_simplified_fix2.copy()

# Set to store indices of duplicate edges to remove
duplicate_removal_indices = set()

# --- Helper function: Normalize edge as unordered pair of start/end points ---
def normalize_edge(pt1, pt2):
    return tuple(sorted([tuple(pt1), tuple(pt2)]))

# --- Step 1: Loop through roundabouts and their connected streets ---
for ridx, streets in roundabout_to_streets.items():
    # Collect all edges from the connected streets
    street_edges = gdf_simplified_fix3[gdf_simplified_fix3['name'].isin(streets)]

    # Group edges by normalized endpoint pair (to catch reversed duplicates)
    edge_groups = defaultdict(list)

    for idx, row in street_edges.iterrows():
        coords = list(row.geometry.coords)
        edge_key = normalize_edge(coords[0], coords[-1])
        edge_groups[edge_key].append((idx, row.geometry.length))

    # --- Step 2: Within each group, retain only the longest edge ---
    for edges in edge_groups.values():
        if len(edges) > 1:
            # Sort by length in descending order
            edges_sorted = sorted(edges, key=lambda x: -x[1])
            # Keep the longest (first); mark others for removal
            for edge_idx, _ in edges_sorted[1:]:
                duplicate_removal_indices.add(edge_idx)

# --- Step 3: Remove marked duplicates and export result ---
gdf_simplified_fix3 = gdf_simplified_fix3[~gdf_simplified_fix3.index.isin(duplicate_removal_indices)].copy()
gdf_simplified_fix3.to_file(f'{data_folder}/gdf_fix_ra.shp')



In [30]:
## Test for the roundabout
joined = gpd.sjoin(gdf_roundabouts, gdf_simplified_fix3, how='left', predicate='intersects')

def compare_roundabout_dicts(old_dict, new_dict):
    differences = {}

    all_keys = set(old_dict.keys()).union(new_dict.keys())

    for key in all_keys:
        old = set(old_dict.get(key, []))
        new = set(new_dict.get(key, []))

        if old != new:
            differences[key] = {
                'added': new - old,
                'removed': old - new
            }

    return differences
roundabout_to_streets_new = defaultdict(set)

for ridx, row in joined.iterrows():
    roundabout_idx = row.name  # index of the roundabout
    street_name = row['name']
    if pd.notnull(street_name):
        roundabout_to_streets_new[roundabout_idx].add(street_name)
diffs = compare_roundabout_dicts(roundabout_to_streets, roundabout_to_streets_new)

if not diffs:
    print("✅ Roundabout-street mapping is unchanged.")
else:
    print("❌ Differences found:")
    for ridx, change in diffs.items():
        print(f"\nRoundabout {ridx}:")
        if change['added']:
            print(f"  ➕ Streets added: {sorted(change['added'])}")
        if change['removed']:
            print(f"  ➖ Streets removed: {sorted(change['removed'])}")



❌ Differences found:

Roundabout 53:
  ➖ Streets removed: ['Piazza Enrico Morselli']

Roundabout 117:
  ➖ Streets removed: ['Via Druento']

Roundabout 131:
  ➖ Streets removed: ['Via Amedeo di Castellamonte']

Roundabout 210:
  ➖ Streets removed: ['Via Giulio Biglieri']

Roundabout 216:
  ➖ Streets removed: ['Largo Cardinale Massaia']

Roundabout 253:
  ➖ Streets removed: ['Lungo Dora Colletta']


In [31]:
conn_dict2 = build_street_connection_dict(gdf_simplified_fix3)

def compare_connection_dicts(original_connection_dict, updated_connection_dict):
    """
    Compare two street connection dictionaries. Identify connections that existed in the original
    version but are missing in the updated version.

    Parameters:
        original_connection_dict (dict): { (street1, street2): [Point, ...] }
        updated_connection_dict (dict): { (street1, street2): [Point, ...] }

    Returns:
        missing_connections (dict): connections from the original missing in the updated
    """
    missing_connections = {}

    for conn_pair, orig_points in original_connection_dict.items():
        if conn_pair not in updated_connection_dict:
            missing_connections[conn_pair] = orig_points
        else:
            # Optionally, you can check if some intersection points are missing
            updated_coords = {tuple(pt.coords)[0] for pt in updated_connection_dict[conn_pair]}
            missing_pts = [pt for pt in orig_points if tuple(pt.coords)[0] not in updated_coords]
            if missing_pts:
                missing_connections[conn_pair] = missing_pts

    return missing_connections

# Example usage
missing = compare_connection_dicts(conn_dict, conn_dict2 )

# Print or log missing entries
for pair, pts in missing.items():
    print(f"Missing connection between {pair[0]} and {pair[1]} at {len(pts)} point(s)")


100%|██████████| 31624/31624 [00:02<00:00, 10884.96it/s]


Missing connection between Via Ermanno Fenoglietti and Via Nizza at 1 point(s)
Missing connection between Via Finalmarina and Via Nizza at 1 point(s)
Missing connection between Via Lavagna and Via Nizza at 1 point(s)
Missing connection between Via Giulio Biglieri and Via Nizza at 1 point(s)
Missing connection between Via Carlo Bianco and Via Giovanni Servais at 1 point(s)
Missing connection between Corso Sacco e Vanzetti and Via Giovanni Servais at 1 point(s)
Missing connection between Via Santa Maria Mazzarello and Via Villa Giusti at 1 point(s)
Missing connection between Via Santa Maria Mazzarello and Via Valsugana at 1 point(s)
Missing connection between Via Monginevro and Via Santa Maria Mazzarello at 2 point(s)
Missing connection between Via Francesco De Sanctis and Via Santa Maria Mazzarello at 2 point(s)
Missing connection between Via Guido Reni and Via Santa Maria Mazzarello at 3 point(s)
Missing connection between Corso Regina Margherita and Via Antonio Fontanesi at 3 point(s)

In [32]:
def filter_intersection_points(points, tolerance=70):
    """
    Filters a list of intersection points based on spatial proximity.

    Logic:
    - If only 1 or 0 points: return as-is with empty representative mapping.
    - If 2 points: if close (< tolerance), return one and map both to it.
    - If >2: use DBSCAN to group points, select one per group, and store mapping from all grouped points to representative.

    Ensures the representative point is not duplicated in its own value list.

    Parameters:
        points (list of shapely.geometry.Point)
        tolerance (float): distance threshold in meters for clustering

    Returns:
        tuple:
            - list of representative points (filtered)
            - dict: {representative Point: [all merged Point(s) excluding rep]}
    """
    if len(points) <= 1:
        return points, {}

    if len(points) == 2:
        if points[0].distance(points[1]) < tolerance:
            return [points[0]], {points[0]: [points[1]]}
        else:
            return points, {}

    # More than two points → use DBSCAN
    coords = np.array([[pt.x, pt.y] for pt in points])
    db = DBSCAN(eps=tolerance-10, min_samples=1).fit(coords)
    labels = db.labels_

    result = []
    group_map = defaultdict(list)

    label_to_rep = {}
    for label in np.unique(labels):
        rep_idx = np.where(labels == label)[0][0]
        rep = points[rep_idx]
        label_to_rep[label] = rep
        result.append(rep)

    for pt, label in zip(points, labels):
        rep = label_to_rep[label]
        if pt != rep:
            group_map[rep].append(pt)

    return result, group_map


In [33]:

def find_nearest_line_from_list(lines: list, point: Point):
    tree = STRtree(lines)
    nearest_geom = tree.nearest(point)
    return nearest_geom

def get_closest_endpoint_to_line(line_a: LineString, line_b: LineString):
    """Returns the endpoint of line_a closest to line_b and its distance."""
    endpoints = [Point(line_a.coords[0]), Point(line_a.coords[-1])]
    distances = [pt.distance(line_b) for pt in endpoints]
    return np.argmin(distances),distances[np.argmin(distances)]

def extend_line_to_target(line: LineString, endpoint_index: int, target: LineString) -> LineString:
    """
    Extend a line by projecting its start or end point onto a target line.

    Parameters:
        line (LineString): The line to extend.
        endpoint_index (int): Use 0 to extend from start, -1 from end.
        target (LineString): The line to project the endpoint onto.

    Returns:
        LineString: Extended line.
    """
    coords = list(line.coords)

    if len(coords) < 2:
        return line  # Can't extend a single-point line

    # Get reference segment
    if endpoint_index == 0:
        p2 = Point(coords[0])
    else:
        p2 = Point(coords[-1])

    # Get projected point from p2 to target
    proj_distance = target.project(p2)
    projected_point = target.interpolate(proj_distance)

    # Extend line to this projected point
    if endpoint_index == 0:
        new_coords = [projected_point.coords[0]] + coords
    else:
        new_coords = coords + [projected_point.coords[0]]

    return LineString(new_coords)

def connect_lines(line1, line2):
    """
    Returns a modified line1 or line2 extended to intersect with the other.
    """
    lg1  = line1.geometry
    lg2 = line2.geometry
    idx1, dist1 = get_closest_endpoint_to_line(lg1, lg2)
    idx2, dist2 = get_closest_endpoint_to_line(lg2, lg1)

    if dist1 < dist2:
        return extend_line_to_target(lg1, idx1,lg2), line1.name

    else:
        return extend_line_to_target(lg2, idx2,lg1), line2.name

def connect_nearest_lines(lines1, lines2, point, gdf_streets):
    """
    Finds the nearest line in each set to a given point, connects them using a custom function,
    and updates the geometry in gdf_streets.

    Parameters:
        lines1 (GeoDataFrame): First set of candidate lines (subset of gdf_streets).
        lines2 (GeoDataFrame): Second set of candidate lines (subset of gdf_streets).
        point (shapely.geometry.Point): The reference point to determine nearest lines.
        gdf_streets (GeoDataFrame): The full street dataset to be updated.
        connect_lines_func (function): A function that takes two lines and returns (new_line, index_to_update).

    Returns:
        GeoDataFrame: Updated gdf_streets with the new connected line geometry.
    """

    def get_nearest_line(lines, pt):
        return lines.iloc[find_nearest_line_from_list(list(lines.geometry), pt)] if len(lines) > 1 else lines.iloc[0]

    line1 = get_nearest_line(lines1, point)
    line2 = get_nearest_line(lines2, point)
    new_line,new_idx = connect_lines(line1,line2)
    gdf_streets.at[new_idx, 'geometry'] = new_line
    return gdf_streets



In [34]:

def find_nearest_intersections_overlay(gdf1, gdf2, ref_pnt):
    """
    Vectorized approach: find the minimum distance from each gdf1 line to the nearest intersection
    with lines in gdf2, measured from a reference point on each gdf1 line.

    Parameters:
        gdf1 (GeoDataFrame): GeoDataFrame with LineString geometries.
        gdf2 (GeoDataFrame): Another GeoDataFrame with LineString geometries.
        ref (str): Reference point on gdf1 geometry. Options: 'centroid', 'start', or 'end'.

    Returns:
        list of float: Distances from each gdf1 feature to its nearest intersection point, or 35 if no intersection.
        :param ref_pnt:
    """

    gdf1['geometry'] = gdf1['geometry'].buffer(0.1)
    gdf2['geometry'] =  gdf1['geometry'].buffer(0.1)
    intersections = gpd.overlay(gdf1, gdf2, how='intersection')


    if intersections.empty:
        return 35
    # Map reference points to intersections
        # Compute distance to a given point (pt_coords should be a shapely.geometry.Point)
    intersections['distance'] = intersections.geometry.distance(ref_pnt)
    return intersections['distance'].min()


In [35]:

TOLERANCE_NEAREST = 30

# Main loop
def snap_missing_points_to_streets(gdf_streets, missing):
    for (street1, street2), points_0 in tqdm(missing.items(), total=len(missing)):

        points, points_map = filter_intersection_points(points_0)
        lines1 = gdf_streets[gdf_streets['name'] == street1].copy()
        lines2 = gdf_streets[gdf_streets['name'] == street2].copy()
        if len(points)==1:
            if len(lines1.sjoin(lines2))>0:
                continue
            else:
                gdf_streets= connect_nearest_lines(lines1, lines2, points[0], gdf_streets)
        else:
            for pt in points:
                if find_nearest_intersections_overlay(lines1.copy(),lines2.copy(),pt) < TOLERANCE_NEAREST:
                    continue
                else:
                    gdf_streets= connect_nearest_lines(lines1.copy(), lines2.copy(), pt, gdf_streets)
    return gdf_streets

gdf_fixed = snap_missing_points_to_streets(gdf_simplified_fix3.copy(), missing)
gdf_fixed.reset_index().to_file(f'{data_folder}/network_final.shp')


 62%|██████▏   | 2271/3663 [00:21<00:13, 105.00it/s]


IndexError: single positional indexer is out-of-bounds

In [234]:


def merge_lines_by_attributes(gdf, attrs=['name', 'highway']):
    """
    Fast merge lines grouped by given attributes.
    """
    records = []
    groupby = gdf.groupby(attrs, sort=False)
    for key, group in tqdm(groupby,total=len(groupby)):
        lines = list(group.geometry)

        if len(lines) == 1:
            records.append({**dict(zip(attrs, key)), 'geometry': lines[0]})
            continue

        merged = linemerge(MultiLineString(lines))

        if isinstance(merged, LineString):
            records.append({**dict(zip(attrs, key)), 'geometry': merged})
        elif isinstance(merged, MultiLineString):
            for part in merged.geoms:
                records.append({**dict(zip(attrs, key)), 'geometry': part})

    return gpd.GeoDataFrame(records, crs=gdf.crs)


def extract_internal_intersections_spatial_index(geoms):
    geoms = [g for g in geoms if isinstance(g, BaseGeometry) and not g.is_empty]
    tree = STRtree(geoms)
    all_points = set()
    i=0
    for geom in tqdm(geoms,total=len(geoms)):

        candidates_0 = tree.query(geom)
        candidates = [geoms[i] for i in range(len(geoms)) if i in candidates_0]
        i+=1
        for other in candidates:
            if other is geom or not isinstance(other, BaseGeometry):
                continue
            if not geom.intersects(other):
                continue

            inter = geom.intersection(other)
            if inter.is_empty:
                continue

            if inter.geom_type == 'Point':
                all_points.add(inter)
            elif inter.geom_type.startswith('Multi') or inter.geom_type == 'GeometryCollection':
                all_points.update(
                    g for g in inter.geoms if g.geom_type == 'Point'
                )

    return list(all_points)





def split_lines_by_intersections_fast(gdf, intersections, buffer_eps=0.01):
    """
    Efficiently split line geometries by intersection points, preserving attributes.
    """
    if not intersections:
        return gdf.copy()

    # Use GeoSeries instead of list of geometries where possible
    splitter_union = unary_union([pt.buffer(buffer_eps).boundary for pt in intersections])
    splitter_union_prep = prep(splitter_union)  # speed up repeated intersection tests

    records = []
    get_attr = lambda row, attr: getattr(row, attr, None)

    for row in gdf.itertuples(index=False):
        geom = row.geometry

        if splitter_union_prep.intersects(geom):
            try:
                result = split(geom, splitter_union)
                records.extend({
                    'name': get_attr(row, 'name'),
                    'highway': get_attr(row, 'highway'),
                    'geometry': part
                } for part in result.geoms if part.length > 0)
            except Exception:
                # Fallback if split fails
                records.append({
                    'name': get_attr(row, 'name'),
                    'highway': get_attr(row, 'highway'),
                    'geometry': geom
                })
        else:
            records.append({
                'name': get_attr(row, 'name'),
                'highway': get_attr(row, 'highway'),
                'geometry': geom
            })

    return gpd.GeoDataFrame(records, crs=gdf.crs)






In [235]:
def filter_edges_by_deadend_criteria(gdf_edges, preserved_old,preserved_new, min_length=30):
    """
    Removes edges that meet the following condition:
    - One endpoint is a deadend
    - The line is shorter than `min_length`
    - That deadend is NOT in the `preserved_deadends` set

    Parameters:
        gdf_edges (GeoDataFrame): Input edge layer with LineString geometries.
        preserved_deadends (set of shapely.geometry.Point): Dead-ends to preserve.
        min_length (float): Length threshold below which certain edges may be removed.

    Returns:
        GeoDataFrame: Filtered edge layer with unwanted short deadend connections removed.
    """
    filtered = []

    for idx, row in tqdm(gdf_edges.iterrows(),total=len(gdf_edges)):
        geom = row.geometry
        coords = list(geom.coords)
        p_start = coords[0]
        p_end = coords[-1]

        # Condition: one deadend, short, and that end not in preserved set
        if geom.length < min_length:
            if p_start in preserved_new and p_start not in preserved_old:
                continue
            if p_end  in preserved_new and p_end  not in preserved_old:
                continue

        # Keep otherwise
        filtered.append(row)

    return gpd.GeoDataFrame(filtered, columns=gdf_edges.columns, crs=gdf_edges.crs)
# ----------------------------
# ⚙️ Full Efficient Workflow
# ----------------------------

# Step 1: Merge connected lines with same 'name' and 'highway'
merged_0 = merge_lines_by_attributes(gdf_fixed, attrs=['name', 'highway'])
merged = merged_0[merged_0.geometry.notnull()]

# Step 2: Build internal intersection points efficiently
internal_points = extract_internal_intersections_spatial_index(list(merged.geometry))

# Step 3: Split merged lines by these points
final_result = split_lines_by_intersections_fast(merged, internal_points)


# List of keys with value == 1
preserved_deadends_old= [key for key, value in extract_deadend_points(gdf_edges).items() if value == 1]
preserved_deadends_new= [key for key, value in extract_deadend_points(final_result).items() if value == 1]
new_gdf  =filter_edges_by_deadend_criteria(final_result,preserved_deadends_old,preserved_deadends_new)
new_gdf['length'] = new_gdf.length
new_gdf.reset_index(drop=True).to_file(f'{data_folder}/network_final2.shp')


100%|██████████| 30864/30864 [00:05<00:00, 5550.91it/s]
